In [1]:
import os
from audio_analyzer import AudioAnalyzer
import networkx as nx

# Initialize AudioAnalyzer
analyzer = AudioAnalyzer()

# Load Playlist
playlist_dir = "./playlist"
songs = [os.path.join(playlist_dir, f) for f in os.listdir(playlist_dir) if f.endswith((".mp3", ".wav"))]
if not songs:
    print("No songs found. Please add audio files to the 'playlist/' directory.")
    exit()

# Analyze Songs (BPM and Key)
song_data = []
for song_path in songs:
    features = analyzer.analyze_audio(song_path)
    if features:
        song_data.append({
            "path": song_path,
            "bpm": features.tempo,
            "key": features.key
        })

# Print Analyzed Data
print("Analyzed Songs:")
for song_info in song_data:
    print(f"{os.path.basename(song_info['path'])}: {song_info['bpm']:.2f} BPM, Key: {song_info['key']}")

# Define cost calculation functions
def calculate_bpm_adjustment_cost(bpm1, bpm2):
    """Calculate cost based on how different the BPMs are."""
    return abs(bpm1 - bpm2)

def get_key_compatibility_cost(key1, key2):
    """Calculate cost based on key compatibility using relative and adjacent key information."""
    if key1 == key2:
        return 0  # Same key, no cost
    elif are_relative_keys(key1, key2):
        return 1  # Relative major/minor, low cost
    elif are_adjacent_keys(key1, key2):
        return 2  # Adjacent keys, moderate cost
    else:
        return 5  # Unrelated keys, higher cost

def are_relative_keys(key1, key2):
    """Check if two keys are relative major/minor using a comprehensive dictionary."""
    relative_keys = {
        "C Major": "A Minor", "A Minor": "C Major",
        "G Major": "E Minor", "E Minor": "G Major",
        "D Major": "B Minor", "B Minor": "D Major",
        "A Major": "F# Minor", "F# Minor": "A Major",
        "E Major": "C# Minor", "C# Minor": "E Major",
        "B Major": "G# Minor", "G# Minor": "B Major",
        "F# Major": "D# Minor", "D# Minor": "F# Major",
        "F Major": "D Minor", "D Minor": "F Major",
        "Bb Major": "G Minor", "G Minor": "Bb Major",
        "Eb Major": "C Minor", "C Minor": "Eb Major",
        "Ab Major": "F Minor", "F Minor": "Ab Major",
        "Db Major": "Bb Minor", "Bb Minor": "Db Major"
    }
    return (key1 in relative_keys and relative_keys[key1] == key2)

def are_adjacent_keys(key1, key2):
    """Check if two keys are adjacent on the Camelot Wheel."""
    camelot_wheel = {
        "1A": ["12A", "1B", "2A"],
        "1B": ["12B", "1A", "2B"],
        "2A": ["1A", "2B", "3A"],
        "2B": ["1B", "2A", "3B"],
        # Add additional mappings as required.
    }
    return key2 in camelot_wheel.get(key1, [])

# Create Graph for Transitions
G = nx.DiGraph()
for i, song1 in enumerate(song_data):
    for j, song2 in enumerate(song_data):
        if i != j:
            # Calculate BPM adjustment cost
            bpm_cost = calculate_bpm_adjustment_cost(song1["bpm"], song2["bpm"])
            # Calculate key compatibility cost
            key_cost = get_key_compatibility_cost(song1["key"], song2["key"])
            # Total transition cost (weighted sum: emphasize key compatibility)
            total_cost = bpm_cost + (2 * key_cost)
            # Add edge with the total cost
            G.add_edge(song1["path"], song2["path"], weight=total_cost)

# Build the playlist incrementally
def build_playlist(graph, start_song=None):
    """Build a playlist by always choosing the next song with the lowest transition cost."""
    if start_song is None:
        # Start with the song with the lowest BPM
        start_song = min(song_data, key=lambda x: x["bpm"])["path"]

    playlist = [start_song]
    remaining_songs = set(graph.nodes()) - {start_song}

    while remaining_songs:
        current_song = playlist[-1]
        # Find the next song with the lowest transition cost
        next_song = min(remaining_songs, key=lambda x: graph[current_song][x]["weight"])
        playlist.append(next_song)
        remaining_songs.remove(next_song)

    return playlist

# Build the playlist
playlist_order = build_playlist(G)

# Print the best playlist order
print("\nBest Playlist Order Based on BPM and Key Matching:")
for idx, song_path in enumerate(playlist_order):
    song_info = next(song for song in song_data if song["path"] == song_path)
    print(f"{idx+1}. {os.path.basename(song_path)}: {song_info['bpm']:.2f} BPM, Key: {song_info['key']}")

# Print transition costs between consecutive songs
print("\nTransition Costs:")
for i in range(len(playlist_order) - 1):
    cost = G[playlist_order[i]][playlist_order[i+1]]["weight"]
    print(f"{os.path.basename(playlist_order[i])} -> {os.path.basename(playlist_order[i+1])}: Cost = {cost:.2f}")

C:\Users\Pc\PycharmProjects\MusicDJ\audio_analyzer.py:58: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]  # Use librosa.beat.tempo directly
C:\Users\Pc\PycharmProjects\MusicDJ\audio_analyzer.py:58: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]  # Use librosa.beat.tempo directly
C:\Users\Pc\PycharmProjects\MusicDJ\audio_analyzer.py:58: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)[0]  # Use librosa.beat.tempo directly
C:\Users\P

Analyzed Songs:
Andromedik & Luka - Lied To You.mp3: 114.84 BPM, Key: E Minor
Armin van Buuren - Lose This Feeling (Dimension Remix) .mp3: 114.84 BPM, Key: G Minor
Artemas - I Like The Way You Kiss Me (Culture Shock's D&B Flip) (Free Download).mp3: 114.84 BPM, Key: G# Minor
B-Complex - Beautiful Lies VIP.mp3: 114.84 BPM, Key: D Major
Basstripper - In The City.mp3: 87.59 BPM, Key: D# Minor
Becky Hill, Chase & Status - Disconnect (Live At The BRITs 2024).mp3: 117.45 BPM, Key: F Major
BICEP _ GLUE (Official Video).mp3: 129.20 BPM, Key: G# Minor
Birdy - Wings (Nu_Logic Remix).mp3: 114.84 BPM, Key: G Major
Bou, Hedex, Slay - Closer (Hedex Remix).mp3: 117.45 BPM, Key: A# Minor
Cartoon, Jéja - Why We Lose (feat. Coleman Trapp) _ DnB _ NCS - Copyright Free Music.mp3: 117.45 BPM, Key: E Minor
Cassö x Raye x D Block Europe - Prada (Official Video).mp3: 143.55 BPM, Key: G# Major
Charlotte Plank, Hybrid Minds - Lights (Official Video).mp3: 114.84 BPM, Key: G Minor
Chase & Status - Program ft. IRAH